In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import inception_v3
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, LeakyReLU
from keras.models import Model, load_model
from keras.applications.resnet50 import ResNet50

import keras.backend as K
from keras.models import Sequential
import cv2

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [23]:
os.listdir("../dados")

['all.zip',
 'crop_test',
 'crop_train',
 'model-files',
 'sample_submission.csv',
 'test',
 'test.zip',
 'train',
 'train.csv',
 'train.zip']

In [2]:
train_df = pd.read_csv("../dados/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [4]:
train_df.shape[0]

25361

In [3]:
TRAIN_FOLDER = '../dados/train/'
TEST_FOLDER = '../dados/test/'
bouding = pd.read_csv("bounding_boxes.csv")
load_X = []
load_Y = []

In [5]:
bouding.head(5)

,Image,x0,y0,x1,y1
0,0000e88ab.jpg,223.0,149.0,860.0,438.0
1,0001f9222.jpg,1.0,77.0,744.0,243.0
2,00029d126.jpg,26.0,19.0,1027.0,464.0
3,00050a15a.jpg,17.0,153.0,1012.0,413.0
4,0005c1ef8.jpg,72.0,186.0,879.0,310.0


In [5]:
bouding.loc[bouding['Image'] == '0000e88ab.jpg']

,Image,x0,y0,x1,y1
0,0000e88ab.jpg,223.0,149.0,860.0,438.0


In [23]:
for img in sorted(os.listdir(TRAIN_FOLDER)):
    if img.split('.')[1] in ['jpg', 'jpeg']:
        #try:
        load_X.append(TRAIN_FOLDER + img)
        load_Y.append(train_df[train_df.Image == img].Id.tolist()[0])
        #except:
            #print(img)
            #print(train_df[train_df.Image == img].Id.tolist())

In [24]:
from imgaug import augmenters as iaa
import imgaug as ia

sometimes = lambda aug: iaa.Sometimes(0.5, aug)


seq = iaa.Sequential([
    #sometimes(iaa.Invert(p=1)),
    iaa.ContrastNormalization(alpha=(0.8,1.2)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.2),
    
    iaa.GaussianBlur(sigma=(0, 1.0)),# blur images with a sigma of 0 to 3.0
    iaa.AdditiveGaussianNoise(scale=(0,0.05*225)),
    iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(0, 360), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )
   
 
])

In [25]:
from sklearn.utils import shuffle

class Generator:
    def __init__(self, size, batch_size):
        self.X_train = load_X
        self.Y_train = load_Y 
        self.X_train, self.Y_train = shuffle(self.X_train, self.Y_train)
        self.len = len(self.X_train)
        self.cur_index = 0
        self.size = size
        self.batch_size = batch_size

        self.label_encoder = LabelEncoder()
        integer_encoded = self.label_encoder.fit_transform(self.Y_train)
        # print(integer_encoded)

        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        # print(onehot_encoded)

        self.Y_train = onehot_encoded
        
    def preprocess(self, batch_img_path):
        if K.image_data_format() == 'channels_first':
            imgs = np.ones([self.batch_size, 3, self.size, self.size])
        else:
            imgs = np.ones([self.batch_size, self.size, self.size, 3])
            
        for i, path in enumerate(batch_img_path):
            #try:
            img = cv2.imread(path)
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #adicionar imgaug
            
            #Cropping
            img_name = path.rsplit('/')[1]
            res = bouding.loc[bouding['Image'] == img_name]
            if len(res.y0.tolist()) > 0 and len(res.y1.tolist()) > 0 and len(res.x1.tolist()) > 0 and len(res.x0.tolist()) > 0:
                img = img[int(res.y0.tolist()[0]):int(res.y1.tolist()[0]) , int(res.x0.tolist()[0]):int(res.x1.tolist()[0])]
            
            img = cv2.resize(img, (self.size, self.size))
            img = seq.augment_images(np.array([img]))
            img = img.astype(np.float32) / 255.0

            #if K.image_data_format() == 'channels_first':
            #    img = np.expand_dims(img, 0)
            #else:
            #    img = np.expand_dims(img, -1)

            imgs[i,:,:] = img
            #except:
                #print(img)
                
        return imgs
                
                
    def getBatch(self):
        while True:
            stop_batch = self.cur_index + self.batch_size
            
            if stop_batch <= self.len:
                batch_img_path = self.X_train[self.cur_index: stop_batch]
                X_data = self.preprocess(batch_img_path)
                Y_data = self.Y_train[self.cur_index: stop_batch]
                self.cur_index += self.batch_size
            else:
                self.cur_index = self.cur_index % self.batch_size
                batch_img_path = self.X_train[self.cur_index: (self.cur_index + self.batch_size)]
                X_data = self.preprocess(batch_img_path)
                Y_data = self.Y_train[self.cur_index: (self.cur_index + self.batch_size)]
                self.cur_index += self.batch_size
                
            yield (X_data, Y_data)

In [7]:
import cv2
def prepareImages(data, m, dataset,size):
    print("Preparing images")
    X_train = np.zeros((m, size, size, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        #img = image.load_img("../dados/"+dataset+"/"+fig, target_size=(128, 128, 1))
        #img = Image.open("../dados/"+dataset+"/"+fig)
        #img = img.resize((128, 128), Image.ANTIALIAS)
        #img = img.convert("L")
        #img = cv2.imread("../dados/"+dataset+"/"+fig)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = cv2.resize(img, (64, 64))
        
        imgPath = "../dados/"+dataset+"/"+fig
        
        img = cv2.imread(imgPath)
        img_name = imgPath.rsplit('/')[1]
        res = bouding.loc[bouding['Image'] == img_name]
        if len(res.y0.tolist()) > 0 and len(res.y1.tolist()) > 0 and len(res.x1.tolist()) > 0 and len(res.x0.tolist()) > 0:
            img = img[int(res.y0.tolist()[0]):int(res.y1.tolist()[0]) , int(res.x0.tolist()[0]):int(res.x1.tolist()[0])]
            
        img = cv2.resize(img, (size, size))
        x = image.img_to_array(img)
        x = preprocess_input(x)

        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [8]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    print(y.shape)
    return y, label_encoder

In [16]:
X = prepareImages(train_df, train_df.shape[0], "crop_train")
#X /= 255

Preparing images


TypeError: Cannot cast ufunc subtract output from dtype('float64') to dtype('uint8') with casting rule 'same_kind'

In [9]:
y, label_encoder = prepare_labels(train_df['Id'])

(25361, 5005)


In [28]:
y.shape[1]

5004

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.15, random_state=42)

In [27]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import SGD, Adadelta, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator

opt = SGD(lr=0.0001)
nd = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
Adam(lr=0.01)

In [28]:
train_gen = Generator(197, 14)

In [29]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = (train_gen.size, train_gen.size,3), pooling='no')

In [33]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(32)(x)
x = LeakyReLU()(x)
#x = Conv2D(32, (3, 3), strides = (1, 1), input_shape = (197, 197, 3))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
#x = Activation('relu')(x)
# let's add a fully-connected layer
x = Dense(8)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(1000, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(train_gen.Y_train.shape[1], activation='softmax')(x)

In [34]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [35]:
history = model.fit_generator(train_gen.getBatch(), steps_per_epoch=train_gen.len / train_gen.batch_size, epochs=14, verbose=1, callbacks=[
            EarlyStopping(monitor='val_acc', patience=20, min_delta=0.01, verbose=1),
            ReduceLROnPlateau(monitor='val_acc', patience=3, min_delta=0.1, factor=0.25, min_lr=0.002, verbose=1)
        ])
model.save("novomodel.model")
#gc.collect()

Epoch 1/14
 354/1811 [====>.........................] - ETA: 51:45 - loss: 6.4340 - acc: 0.3822

KeyboardInterrupt: 

In [10]:
test = os.listdir("../dados/test/")
print(len(test))

7960


In [12]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [12]:
test_df.shape[0]

7960

In [13]:
first_df = test_df.iloc[0:2000]
second_df = test_df.iloc[2001:4000]
third_df = test_df.iloc[4001:6000]
forth_df = test_df.iloc[6001:test_df.shape[0]]

In [17]:
forth_df.shape[0]

1959

In [14]:
model = load_model('novomodel.model')

In [17]:
def processByStep(dataFrame, size, first_index):
    X = prepareImages(dataFrame, dataFrame.shape[0], "test", size)
    X /= 255
    predictions = model.predict(np.array(X), verbose=1)
    for i, pred in enumerate(predictions):
        test_df.loc[i + first_index, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [18]:
processByStep(first_df, 197, 0)
processByStep(second_df, 197, 2001)
processByStep(third_df, 197, 4001)
processByStep(forth_df, 197, 6001)

Preparing images
Processing image:  1 ,  00028a005.jpg
Processing image:  501 ,  0f9219c40.jpg
Processing image:  1001 ,  1ea95c247.jpg
Processing image:  1501 ,  2fe4ec2d5.jpg
2000/2000 [==============================] - 107s 54ms/step
Preparing images
Processing image:  1 ,  4043c4a1d.jpg
Processing image:  501 ,  4fe483e96.jpg
Processing image:  1001 ,  6005a13de.jpg
Processing image:  1501 ,  70438d9d1.jpg
1999/1999 [==============================] - 102s 51ms/step
Preparing images
Processing image:  1 ,  802330ff6.jpg
Processing image:  501 ,  906b3c9c3.jpg
Processing image:  1001 ,  a0d962939.jpg
Processing image:  1501 ,  b1aa98e8c.jpg
1999/1999 [==============================] - 100s 50ms/step
Preparing images
Processing image:  1 ,  c0bbbf064.jpg
Processing image:  501 ,  cfeae6723.jpg
Processing image:  1001 ,  e00b17e08.jpg
Processing image:  1501 ,  f1b73b091.jpg
1959/1959 [==============================] - 102s 52ms/step


In [47]:
X = prepareImages(test_df, test_df.shape[0], "test", 197)
X /= 255

Preparing images
Processing image:  1 ,  00028a005.jpg
Processing image:  501 ,  0f9219c40.jpg
Processing image:  1001 ,  1ea95c247.jpg
Processing image:  1501 ,  2fe4ec2d5.jpg
Processing image:  2001 ,  404206f77.jpg
Processing image:  2501 ,  4fc69fd82.jpg
Processing image:  3001 ,  5ffc2d300.jpg
Processing image:  3501 ,  7042be267.jpg
Processing image:  4001 ,  801c6a722.jpg
Processing image:  4501 ,  906aa36c8.jpg
Processing image:  5001 ,  a0d337cdc.jpg
Processing image:  5501 ,  b1a4e4c18.jpg
Processing image:  6001 ,  c0b378682.jpg
Processing image:  6501 ,  cfe9d9d04.jpg
Processing image:  7001 ,  e00620fc3.jpg
Processing image:  7501 ,  f1b504951.jpg


In [11]:
model = load_model('novomodel.model')

In [12]:
predictions = model.predict(np.array(X), verbose=1)

MemoryError: 

In [ ]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [19]:
test_df.to_csv('submission_resnet50.csv', index=False)

In [20]:
test_df.head(10)

,Image,Id
0,00028a005.jpg,new_whale w_81d4bd6 w_69b5ace w_c01c1f4 w_9a9a2c8
1,000dcf7d8.jpg,new_whale w_81d4bd6 w_69b5ace w_c01c1f4 w_04003e9
2,000e7c7df.jpg,new_whale w_81d4bd6 w_69b5ace w_89f521e w_9a9a2c8
3,0019c34f4.jpg,new_whale w_81d4bd6 w_69b5ace w_04003e9 w_df32eec
4,001a4d292.jpg,new_whale w_81d4bd6 w_69b5ace w_c01c1f4 w_9a9a2c8
5,00247bc36.jpg,new_whale w_81d4bd6 w_69b5ace w_89f521e w_9a9a2c8
6,0027089a4.jpg,new_whale w_81d4bd6 w_69b5ace w_89f521e w_9a9a2c8
7,002de4d94.jpg,new_whale w_81d4bd6 w_69b5ace w_04003e9 w_df32eec
8,002f52f0c.jpg,new_whale w_81d4bd6 w_69b5ace w_df32eec w_c01c1f4
9,002fd89d4.jpg,new_whale w_81d4bd6 w_69b5ace w_c01c1f4 w_04003e9


In [22]:
train_df.Id.value_counts()

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
w_700ebb4      57
w_3de579a      54
w_564a34b      51
w_fd3e556      50
w_88e4537      49
w_2b069ba      48
w_d405854      47
w_f0fe284      45
w_789c969      45
w_343f088      40
w_778e474      40
w_5e8e218      40
w_5a2634c      37
w_60ce6fc      37
w_a9304b9      37
w_6822dbc      36
w_af367c3      35
w_f765256      34
w_1ca9ab1      34
w_17b0d3a      33
w_d72771c      32
w_8c25681      31
w_6cda039      31
w_08630fd      31
w_51fc1fc      30
             ... 
w_d609d41       1
w_d44dfdc       1
w_f08d3f7       1
w_a1e3bc9       1
w_567d88c       1
w_536beda       1
w_eb31833       1
w_5ce4848       1
w_6c2e008       1
w_214e081       1
w_b8b8383       1
w_1d2ed91       1
w_808d40b       1
w_2623b69       1
w_8261f4d       1
w_490d198       1
w_6d1f159       1
w_2757c07       1
w_fd4376a       1
w_9e020d1       1
w_2ac6611       1
w_36f91ad       1
w_6fe3566       1
w_795a6b8       1
w_d31f7e6 